In [5]:
D <- read.csv("DST_BIL54.csv")
str(D)
# See the help
# ?strftime
D$time <- as.POSIXct(paste0(D$time,"-01"), "%Y-%m-%d", tz="UTC")


## Year to month for each of them
D$year <- 1900 + as.POSIXlt(D$time)$year + as.POSIXlt(D$time)$mon / 12
class(D$time)
## Make the output variable a floating point (i.e.\ decimal number)
D$total <- as.numeric(D$total) / 1E6

## Divide intro train and test set
teststart <- as.POSIXct("2024-01-01", tz="UTC")
Dtrain <- D[D$time < teststart, ]
Dtest <- D[D$time >= teststart, ]

'data.frame':	84 obs. of  13 variables:
 $ time        : chr  "2018-01" "2018-02" "2018-03" "2018-04" ...
 $ total       : int  2930483 2934044 2941422 2951498 2963093 2973721 2976096 2985537 2983209 2982473 ...
 $ gasoline    : int  1781826 1783338 1787219 1793554 1799795 1805170 1806338 1810615 1808496 1806710 ...
 $ diesel      : int  1136656 1138414 1141576 1144846 1149840 1154651 1155420 1159907 1159418 1160162 ...
 $ fgas        : int  12 12 12 12 12 12 12 12 11 11 ...
 $ ngas        : int  256 255 257 257 256 262 262 261 261 263 ...
 $ electricity : int  9304 9343 9399 9548 9665 9807 9853 10002 10120 10236 ...
 $ petroleum   : int  0 0 0 0 0 0 0 0 0 0 ...
 $ hydrogen    : int  81 80 77 77 77 77 77 80 82 82 ...
 $ methanol    : int  0 0 0 0 0 0 0 0 0 0 ...
 $ ethanol     : int  0 0 0 0 0 0 0 0 0 0 ...
 $ pluginhybrid: int  2340 2593 2873 3195 3439 3734 4126 4652 4812 5000 ...
 $ otherfuels  : int  8 9 9 9 9 8 8 8 9 9 ...


[1] "POSIXct" "POSIXt"